In [1]:
import sys
import datetime
sys.path.append('./formulas')
sys.path.append('./data')

from optionchain import option_chain
from option import option_parser
from blackscholes import call, put

In [2]:
def chain_processor(t, ticker):
    calls = t[0]
    puts = t[1]
    beta = t[2] or 1
    price = t[3]
    calls_table = {}
    for table in calls:
        for index, row in table.iterrows():
            meta = option_parser(row['contractSymbol'], ticker)
            last_price = max(row['lastPrice'], row['ask'])
            year, month, day, instr_type, strike = meta
            time = (datetime.datetime(int(f"20{year}"), int(month), int(day)) - datetime.datetime.now()).days / 365
            if instr_type == 'C':
                if str(year)+str(month)+str(day) not in calls_table:
                    calls_table[str(year)+str(month)+str(day)] = {}
                calls_table[str(year)+str(month)+str(day)][strike] = call(price, strike, 0.05, beta, time), last_price

    puts_table = {}
    for table in puts:
        for index, row in table.iterrows():
            meta = option_parser(row['contractSymbol'], ticker)
            last_price = max(row['lastPrice'], row['ask'])
            year, month, day, instr_type, strike = meta
            time = (datetime.datetime(int(f"20{year}"), int(month), int(day)) - datetime.datetime.now()).days / 365
            if instr_type == 'P':
                if str(year)+str(month)+str(day) not in puts_table:
                    puts_table[str(year)+str(month)+str(day)] = {}
                puts_table[str(year)+str(month)+str(day)][strike] = put(price, strike, 0.05, beta, time), last_price

    # Find the cheapest straddles
    straddles = {}
    for date in calls_table:
        straddles[date] = {}
        for strike in calls_table[date]:
            if date in puts_table:
                if strike in puts_table[date]:
                    straddles[date][strike] = calls_table[date][strike][0] + puts_table[date][strike][0], calls_table[date][strike][1] + puts_table[date][strike][1]


    # Formatting
    slist = []
    for date in straddles:
        for strike in straddles[date]:
            percent_diff = (straddles[date][strike][0] - straddles[date][strike][1]) / straddles[date][strike][1]
            predicted = straddles[date][strike][0]
            slist.append((date, strike, straddles[date][strike][0], straddles[date][strike][1], percent_diff))
    slist.sort(key=lambda x: -x[4])

    return slist

In [3]:
tickers = open("/Users/andrew/Projects/arbitrage-finance/tst.txt", "r").read().splitlines()
name = "tsx3" #tickers[0]
oc_ = []
with open(name + 'rusful.csv', 'w') as f:
    with open(name + 'rusfar.csv', 'w') as f2:
        f.write("ticker,date,strike,predicted,actual,percent_diff")
        f2.write("ticker,date,strike,predicted,actual,percent_diff")
        # New line
        f.write("\n")
        f2.write("\n")
        for ticker in tickers:
            ticker = ticker
            try:
                c = option_chain(ticker)
                oc_.append((c, ticker))
                tlist = chain_processor(c, ticker)
                for t in tlist:
                    f.write(f"{ticker},{t[0]},{t[1]},{t[2]},{t[3]},{t[4]}")
                    # New line
                    f.write("\n")
                    if t[0][:2] != '22':
                        f2.write(f"{ticker},{t[0]},{t[1]},{t[2]},{t[3]},{t[4]}")
                        # New line
                        f2.write("\n")
                
                print(f"Finished {ticker}")
                

            except:
                print(f'Failed to get option chain for {ticker}')


Finished RY
Failed to get option chain for TD
Finished ENB
- CNR: No summary info found, symbol may be delisted
Failed to get option chain for CNR
Finished CP


In [20]:
calls = t[0]
puts = t[1]
beta = t[2] or 1
price = t[3]
calls_table = {}
for table in calls:
    for index, row in table.iterrows():
        meta = option_parser(row['contractSymbol'], ticker)
        last_price = row['lastPrice']
        year, month, day, instr_type, strike = meta
        time = (datetime.datetime(int(f"20{year}"), int(month), int(day)) - datetime.datetime.now()).days / 365
        if instr_type == 'C':
            if str(year)+str(month)+str(day) not in calls_table:
                calls_table[str(year)+str(month)+str(day)] = {}
            calls_table[str(year)+str(month)+str(day)][strike] = call(price, strike, 0.05, beta, time), last_price

puts_table = {}
for table in puts:
    for index, row in table.iterrows():
        meta = option_parser(row['contractSymbol'], ticker)
        last_price = row['lastPrice']
        year, month, day, instr_type, strike = meta
        time = (datetime.datetime(int(f"20{year}"), int(month), int(day)) - datetime.datetime.now()).days / 365
        if instr_type == 'P':
            if str(year)+str(month)+str(day) not in puts_table:
                puts_table[str(year)+str(month)+str(day)] = {}
            puts_table[str(year)+str(month)+str(day)][strike] = put(price, strike, 0.05, beta, time), last_price

# Find the cheapest straddles
straddles = {}
for date in calls_table:
    straddles[date] = {}
    for strike in calls_table[date]:
        if date in puts_table:
            if strike in puts_table[date]:
                straddles[date][strike] = calls_table[date][strike][0] + puts_table[date][strike][0], calls_table[date][strike][1] + puts_table[date][strike][1]


# Find the cheapest straddles
slist = []
for date in straddles:
    for strike in straddles[date]:
        percent_diff = (straddles[date][strike][0] - straddles[date][strike][1]) / straddles[date][strike][1]
        predicted = straddles[date][strike][0]
        slist.append((date, strike, straddles[date][strike][0], straddles[date][strike][1], percent_diff))
slist.sort(key=lambda x: -x[4])
# everything that is not in 2022
leaplist = [x for x in slist if x[0][:2] != '22']

NameError: name 't' is not defined

In [ ]:
for i in range(max(30, len(leaplist))-1):
    print("Date:", leaplist[i][0], "Strike:", leaplist[i][1], "Black Scholes pricing:", leaplist[i][2], "Last Price:", leaplist[i][3], "Percent Difference:", leaplist[i][4]*100, "%")

In [ ]:
for l in slist:
    print(l)